In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

# Configuración de autenticación
client_id = 'b6841f7c97974c8aacbdf448dae8677f'
client_secret = 'bc15cade85bc41f88d451fbb2c832eff'
redirect_uri = 'http://localhost:8888/callback/'
scope = 'user-read-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope=scope
))

# Buscar un artista por nombre
artist_name = 'Kendrick Lamar'  # Reemplaza con el nombre del artista
results = sp.search(q=f'artist:{artist_name}', type='artist')

# Obtener el ID del primer artista en los resultados
if not results['artists']['items']:
    print("Artista no encontrado.")
else:
    artist_id = results['artists']['items'][0]['id']

    # Obtener todos los álbumes del artista
    albums = sp.artist_albums(artist_id, album_type='album', limit=50)

    # Lista para almacenar todas las canciones y álbumes
    all_songs = [
        {
            'album': album['name'],
            'track_name': track['name'],
            'track_id': track['id'],
            'duration_sec': track['duration_ms'] / 1000,
            'popularity': track['popularity'],  # Información ya disponible en el objeto track
            'release_date': album['release_date'],
            'album_image_url': album['images'][0]['url'] if album['images'] else None
        }
        for album in albums['items']
        for track in sp.album_tracks(album['id'])['items']
    ]

    # Crear un DataFrame con todos los datos de las canciones
    df = pd.DataFrame(all_songs)

    # Añadir el nombre del artista al DataFrame
    df['artist'] = artist_name

    # Mostrar las primeras filas del DataFrame para verificar
    print(df.head())

In [21]:
df.to_csv(f'{artist_name}.csv')